In [1]:
from numba import jit, njit, vectorize
import random

# #@jit(nopython=True)
# def monte_carlo_pi(nsamples):
#     acc = 0
#     for i in range(nsamples):
#         x = random.random()
#         y = random.random()
#         if (x ** 2 + y ** 2) < 1.0:
#             acc += 1
#     return 4.0 * acc / nsamples

C:\Users\admin\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\admin\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\admin\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [25]:
%time monte_carlo_pi(10000)

Wall time: 25.6 ms


3.1408

In [26]:
monte_jit=jit()(monte_carlo_pi)

In [28]:
%time monte_jit(10000)

Wall time: 0 ns


3.15

In [29]:
def func(inp):
    out=[]
    for itm in inp:
        if itm%2==0:
            out.append(2)
        else:
            out.append('1')
    return out

In [33]:
test=list(range(int(1e6)))

In [ ]:
import timeit
timeit.timeit("func(test)",globals=globals())

In [37]:
fun=jit()(func)

In [44]:
%time a=fun()

UsageError: %%time is a cell magic, but the cell body is empty. Did you mean the line magic %time (single %)?


In [43]:
from sympy import *
import numpy as np

x, y ,z = symbols('x, y, z')
eq1 = Eq(x-5*y**2-z, 4)
eq2 = Eq(x**2 + y+9*z, 0)
eq3 = Eq(x+y+z,5)

sol = solve([eq1, eq2,eq3], [x, y,z])

In [ ]:
t=[tuple(v.evalf() for v in s) for s in sol]
t

In [17]:
f=[q.evalf() for p in sol for q in p]
f

[-2.56155281280883,
 -2.56155281280883,
 1.56155281280883,
 1.56155281280883,
 -1.30277563773199,
 2.30277563773199,
 2.30277563773199,
 -1.30277563773199]

In [44]:
%%time
s=[list(p.evalf() for p in r ) for r in sol]
np.array(s,dtype=float)

Wall time: 0 ns


array([], dtype=float64)

In [45]:
sol

[]

In [37]:
%%time
np.array(sol,dtype=float)

Wall time: 44.6 ms


array([[-2.56155281, -2.56155281],
       [ 1.56155281,  1.56155281],
       [-1.30277564,  2.30277564],
       [ 2.30277564, -1.30277564]])

In [12]:
#!/usr/bin/env python
# coding: utf-8


import numpy as np
from sympy.abc import c,x,y,u,v,omega,rho,beta
from sympy import symbols

ep1,ep2=symbols("epsilon_1 epsilon_2")
e1=x*(1-x**2-y**2)-omega*y+ep1*u
e2=y*(1-x**2-y**2)+omega*x
e3=rho*v
e4=-rho*beta*u-c*v+ep2*x

# define equation
#@jit(nopython=False)
def evaluate(X,Y,U,V,epsilon1=6,epsilon2=8,b=1,p=1,C=1,w=5):
    ''' input format: X,Y,U,V,epsilon1=6,epsilon2=8,b=1,p=1,C=1,w=5'''
    x_dot = e1.subs([(x,X),(y,Y),(u,U),(v,V),(beta,b),(rho,p),(c,C),(omega,w),(ep1,epsilon1),(ep2,epsilon2)])
    y_dot = e2.subs([(x,X),(y,Y),(u,U),(v,V),(beta,b),(rho,p),(c,C),(omega,w),(ep1,epsilon1),(ep2,epsilon2)])
    u_dot = e3.subs([(x,X),(y,Y),(u,U),(v,V),(beta,b),(rho,p),(c,C),(omega,w),(ep1,epsilon1),(ep2,epsilon2)])
    v_dot = e4.subs([(x,X),(y,Y),(u,U),(v,V),(beta,b),(rho,p),(c,C),(omega,w),(ep1,epsilon1),(ep2,epsilon2)])

    return x_dot.evalf(), y_dot.evalf(), u_dot.evalf(), v_dot.evalf()

#@jit(nopython=False)
def compute(x0,y0,u0,v0,t0,epsilon1=6,epsilon2=8,b=1,p=1,C=1,w=5,steps=1000,delta_t=0.01):
    ''' 
    input format: x0,y0,u0,v0,t0,epsilon1=6,epsilon2=8,b=1,p=1,C=1,w=5,steps=1000,delta_t=0.01
    usage : x1,y1,u1,v1,t1=compute(0.1,0,0,0,t0,p=2,steps=2000)	
    '''
    steps=int(steps)
    # initialize solutions arrays (+1 for initial conditions)
    xx = np.empty((steps + 1))
    yy = np.empty((steps + 1))
    uu = np.empty((steps + 1))
    vv = np.empty((steps + 1))
    tt = np.empty((steps+1))
    # fill in initial conditions
    xx[0], yy[0], uu[0], vv[0] = (x0,y0,u0,v0)
    tt[0] = t0
    # solve equation system

    for i in range(steps):
        # Calculate derivatives
        x_dot, y_dot, u_dot, v_dot = evaluate(xx[i], yy[i], uu[i], vv[i],epsilon1,epsilon2,b,p,C,w)

        tt[i+1] = tt[i] + delta_t

        xx[i + 1] = xx[i] + (x_dot * delta_t)
        yy[i + 1] = yy[i] + (y_dot * delta_t)
        uu[i + 1] = uu[i] + (u_dot * delta_t)
        vv[i + 1] = vv[i] + (v_dot * delta_t)
    return xx,yy,uu,vv,tt

#xaxis=np.linspace(-4,4,)
for P in range(1,2):
    n=1e3
    dt=0.001
    n=int(n)
    x1,y1,u1,v1,t1=compute(0.1,0,0,0,0.0,p=P,steps=n,delta_t=dt)
    summation1=0.0
    summation2=0.0
    k=0
    for i in range(1,n+1):
        k=k+1
        summation1=summation1+ np.log(np.abs((x1[i]-x1[i-1])/dt))
        summation2=summation2+ np.log(np.abs((y1[i]-y1[i-1])/dt))

    lamda=[]
    lamda.append(summation1/k)
    lamda.append(summation1/k)
    print(f"lamda1_p_{P}=", lamda[0])
    print(f"lamda2_p_{P}=", lamda[1])

    np.savetxt(f'x_{P}.txt', (x1))
    np.savetxt(f'y_{P}.txt', (y1))




lamda1_p_1= -1.4460099085225844
lamda2_p_1= -1.4460099085225844
